### Дообучение модели для генерации анекдотов 

Используется модель: Qwen3-0.6B (Base)

In [ ]:
import inspect
import re
import random
from pathlib import Path
from typing import Optional, Tuple, List, Iterable, Union
from tqdm import tqdm

import pandas as pd
import torch

from datasets import Dataset
from peft import LoraConfig, PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig

rng = random.Random(42)


In [ ]:
import kagglehub 

# Download latest version
path = kagglehub.dataset_download("michaelstepanovsky/anecdoted")


100%|██████████| 5.27M/5.27M [00:01<00:00, 4.34MB/s]

Extracting files...


In [ ]:
root = Path(path)
csv_files = sorted(root.rglob("*.csv"), key=lambda p: p.stat().st_size, reverse=True)

print("files: ", csv_files)

dataset = pd.read_csv(csv_files[0])

display(dataset.head(5))

print("Всего анекдотов: ", dataset.shape[0])

files:  [WindowsPath('C:/Users/user/.cache/kagglehub/datasets/michaelstepanovsky/anecdoted/versions/1/anecdotes_dataset.csv')]


,joke,word_count,char_count
0,Нижегородский купчина рассказывает своим друзь...,71,358
1,"- Папа, а марсиане есть?- Нет, сынок, кончилис...",21,78
2,- Где работает ваш муж?- Уже третий месяц на л...,28,126
3,"- Дорогой, я похожа на идеальную женщину ?- Не...",29,111
4,"- О, милая! Панировка этих котлет такая хрустя...",30,144


Всего анекдотов:  44108


In [53]:
# сетапы
with open("data/prefixes.txt", "r", encoding="utf-8") as f:
    prefixes = f.readlines()


print(prefixes[:5])

['1 Идёт мужик по лесу\n', '2 Встречаются два друга\n', '3 Приходят мужик в бар\n', '4 Жена говорит мужу\n', '5 Приходят альфа, бета и гамма в бар\n']


Формализовать задачу можно следующим образом: дан набор сетапов, к которым необходимо дописать панчлайн. Следовательно, необходимо ставить перед моделью задачу "сетап -> панчлайн" -> loss считать на панчлайнах

## Подготовка данных

В анекдоте сетап часто = первая фраза/первая реплика/первые N слов, а панчлайн = всё остальное

Тестовый вход: “Идёт мужик по лесу” (без контекста).

Значит в обучении должны быть такие же короткие входы

Будем собирать анекдоты по следующему шаблону:

    Сетап: <setup>
    Панчлайн: <punchline>

### Подготовка сетапов

In [56]:
def split_id_and_text(s: str) -> Tuple[Optional[int], str]:
    "Разбивает сетапы на id и текст"
    s = s.replace("\r\n", "\n").replace("\r", "\n").strip()
    parts = s.split(" ", 1)
    head, rest = parts[0], parts[1]
    
    return int(head), rest.strip()

In [61]:
setups = [split_id_and_text(p)[1] for p in prefixes]
ids = [split_id_and_text(p)[0] for p in prefixes]

print(setups) 
print(ids)

['Идёт мужик по лесу', 'Встречаются два друга', 'Приходят мужик в бар', 'Жена говорит мужу', 'Приходят альфа, бета и гамма в бар', 'Идёт медведь по лесу', 'Приходит мужик к врачу', 'Встречаются русский, американец и немец', 'Идёт по улице девушка', 'Приходит мужик в магазин', 'Еще сто лет назад', 'Встречаются Вовочка и Петька', 'Идёт по лесу охотник', 'Я хорошо готовлю, стираю и убираю в квартире', 'Жена спрашивает у мужа', 'Сидят в баре два друга', 'Идёт по пустыне караван', 'Приходит мужик в аптеку', 'Встречаются два программиста', '- Послушайте, у этого парня в резюме', 'Приходит мужик в банк', 'Сидят на скамейке два пенсионера', 'Идёт по лесу грибник', 'Приходит мужик в ресторан', '- Я дочитал учебник по теории вероятности', 'Идёт по улице студент', 'Заходит студент в кофейню', 'Сидят в очереди два человека', 'Идёт по лесу шаман', 'Приходит мужик в библиотеку', 'Идёт по улице кот', 'Встречаются два математика', 'Приходит программист в бар', 'Сидит кот на клавиатуре', 'Доказывает те

### Подготовка анекдотов

In [45]:
# шаблоны для разбиения

_SENT_SPLIT_RE = re.compile(r"(?<=[.!?])\s+")
_DASH_RE = re.compile(r"\s*[-—]\s*")

In [59]:
def split_setup_punchline(text: str) -> Optional[Tuple[str, str]]:
    """Функция для разбиения анекдота на сетап/панчлайн
    Args:
        text (str): Анекдот
        
    Returns:
        Optional[Tuple[str, str]]: Пару (сетап, панчлайн). Если разбить анекдот не получилось, возвращает None"""
    t = text.strip()
    if not t:
        return None

    # 1) Если многострочный — первая строка сетап
    if "\n" in t:
        lines = [x.strip() for x in t.split("\n") if x.strip()]
        if len(lines) >= 2:
            setup = lines[0]
            punch = " ".join(lines[1:]).strip()
            if len(setup) >= 8 and len(punch) >= 8:
                return setup, punch

    # 2) Если есть явный диалог/тире — берём кусок до первого "—" как сетап (часто работает)
    if "—" in t or " - " in t:
        parts = _DASH_RE.split(t, maxsplit=1)
        if len(parts) == 2:
            setup = parts[0].strip()
            punch = ("— " + parts[1].strip()).strip()
            if len(setup) >= 8 and len(punch) >= 8:
                return setup, punch

    # 3) По первой фразе
    sents = _SENT_SPLIT_RE.split(t)
    if len(sents) >= 2:
        setup = sents[0].strip()
        punch = " ".join(sents[1:]).strip()
        if len(setup) >= 8 and len(punch) >= 8:
            return setup, punch

    # 4) По словам (fallback)
    words = t.split()
    if len(words) >= 12:
        k = min(10, max(5, len(words) // 3))
        setup = " ".join(words[:k])
        punch = " ".join(words[k:])
        if len(setup) >= 8 and len(punch) >= 8:
            return setup, punch

    return None

def make_prefix_splits(text: str, n_variants: int = 2):
    """Делаем несколько вариантов: сетап = первые k слов, панчлайн = хвост."""
    t = text.strip()
    words = t.split()
    out = []
    if len(words) < 14:
        return out
    for _ in range(n_variants):
        k = random.randint(4, min(12, len(words) - 8))
        setup = " ".join(words[:k])
        punch = " ".join(words[k:])
        if len(setup) >= 8 and len(punch) >= 8:
            out.append((setup, punch))
    return out

def canonicalize_setup(setup: str, setups: List[str]):
    """Если сетап похож на один из тестовых — приводим к канонической форме."""
    s = setup.strip().lower()
    for canon in setups:
        c = canon.lower()
        if s.startswith(c):
            return canon
    return setup

In [ ]:
pairs = []
for t in dataset["joke"].tolist():
    sp = split_setup_punchline(t)
    if sp is not None:
        setup, punch = sp
        setup = canonicalize_setup(setup, setups)
        pairs.append((setup, punch))

    for setup, punch in make_prefix_splits(t, n_variants=2):
        setup = canonicalize_setup(setup, setups)
        pairs.append((setup, punch))

In [ ]:
pairs = list(dict.fromkeys(pairs)) # drop duplicates

print("Сформировано пар (setup, punchline):", len(pairs))

pairs[:10]

Сформировано пар (setup, punchline): 111604


[('Нижегородский купчина рассказывает своим друзьям',
  '— купцам:- Не люблю я англичан! Сумасшедший народ! Я в Питере был, в гостинице по соседству с англичанином жил - никакого житья: без конца то горло полощет, то вообще неприличные звуки издаёт..- А вы уверены, что это был англичанин?- Ну как же! У него на двери и табличка висела с именем-фамилией: Уотер Клозет.'),
 ('Нижегородский купчина рассказывает своим друзьям-купцам:- Не люблю я англичан! Сумасшедший народ! Я',
  'в Питере был, в гостинице по соседству с англичанином жил - никакого житья: без конца то горло полощет, то вообще неприличные звуки издаёт..- А вы уверены, что это был англичанин?- Ну как же! У него на двери и табличка висела с именем-фамилией: Уотер Клозет.'),
 ('Нижегородский купчина рассказывает своим друзьям-купцам:- Не',
  'люблю я англичан! Сумасшедший народ! Я в Питере был, в гостинице по соседству с англичанином жил - никакого житья: без конца то горло полощет, то вообще неприличные звуки издаёт..- А вы уве

### Подготовка к обучению

In [ ]:
def norm(s: str) -> str:
    return str(s).replace("\r\n", "\n").replace("\r", "\n").strip()

def build_prompt(setup: str) -> str:
    # только сетап
    s = norm(setup)
    return s

def build_completion(punchline: str) -> str:
    # Ведущий пробел помогает сделать границу prompt|completion стабильной
    # и избегает “склейки” слов, если сетап без пробела на конце.
    c = norm(punchline)
    return " " + c

In [132]:
rows = []
for setup, punch in pairs:
    p = build_prompt(setup)
    c = build_completion(punch)
    if len(p) == 0 or len(c.strip()) == 0:
        continue
    rows.append({"prompt": p, "completion": c})

pc_ds = Dataset.from_list(rows).shuffle(seed=42)
splits = pc_ds.train_test_split(test_size=0.01, seed=42)
train_pc, eval_pc = splits["train"], splits["test"]

In [133]:
train_pc[18]

{'prompt': 'Приходит Василий Иванович домой пьяный в дым, а Петька сидит дома трезвый, злой Думает:',
 'completion': ' — Ну ладно, попомню я тебе это Взял пластилин и Чапаю второй член из него вылепил, прилепил и спать лег Среди ночи Петька просыпается от истошного крика и понять ничего не может, забыл-то уже, что ночью вытворил А Чапай сидит посреди комнаты и орет: - Петька, пить бросаю, не поверишь, просыпаюсь, а у меня два члена Я один оторвал, а второй сам отпал.'}

### Обучение

In [134]:
MODEL_NAME = "Qwen/Qwen3-0.6B-Base"

In [135]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True


In [136]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

use_cuda = torch.cuda.is_available()
bf16_ok = use_cuda and torch.cuda.is_bf16_supported()
dtype = torch.bfloat16 if bf16_ok else (torch.float16 if use_cuda else torch.float32)

In [137]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, dtype=dtype)

In [138]:
model.gradient_checkpointing_enable()
model.config.use_cache = False

In [139]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)

In [ ]:
cfg = dict(
    output_dir="qwen3-punchlines-sft",
    max_length=256,                
    packing=False,                  
    completion_only_loss=True,      

    num_train_epochs=2,
    learning_rate=1e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",

    per_device_train_batch_size=8 if use_cuda else 1,
    gradient_accumulation_steps=2 if use_cuda else 4,

    logging_steps=20,
    save_strategy="steps",
    save_steps=250,
    eval_strategy="steps",
    eval_steps=250,
    save_total_limit=2,

    report_to=[],
    bf16=(dtype == torch.bfloat16),
    fp16=(dtype == torch.float16),
    max_grad_norm=1.0,
    group_by_length=True,
)

In [141]:
sig = set(inspect.signature(SFTConfig.__init__).parameters.keys())
cfg = {k: v for k, v in cfg.items() if k in sig}

args = SFTConfig(**cfg)


In [142]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_pc,
    eval_dataset=eval_pc,
    peft_config=peft_config,
    processing_class=tokenizer,
)


Truncating eval dataset: 100%|██████████| 1117/1117 [00:00<00:00, 557079.38 examples/s]


In [143]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
250,2.805200,2.913895,2.943386,338136.000000,0.415383
500,2.863600,2.844347,2.874017,682057.000000,0.424938
750,2.675100,2.759075,2.838589,1025786.000000,0.439222
1000,2.767300,2.713129,2.786627,1373362.000000,0.446107
1250,2.622400,2.666569,2.754129,1717359.000000,0.453535
1500,2.656800,2.639953,2.649875,2055058.000000,0.459072
1750,2.599500,2.611976,2.635431,2392357.000000,0.462832
2000,2.555100,2.582747,2.625834,2728120.000000,0.466720
2250,2.503300,2.551079,2.577734,3071381.000000,0.473471
2500,2.580400,2.520362,2.584360,3411977.000000,0.478970


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen3-0.6B-Base/resolve/main/config.json (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1007)')))"), '(Request ID: 7e05879f-8ab4-437b-a9f9-477c65144285)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-0.6B-Base/resolve/main/config.json
Retrying in 1s [Retry 1/5].


TrainOutput(global_step=13812, training_loss=2.1147494690334696, metrics={'train_runtime': 9879.8021, 'train_samples_per_second': 22.366, 'train_steps_per_second': 1.398, 'total_flos': 5.283486875477606e+16, 'train_loss': 2.1147494690334696, 'entropy': 2.3177488523980845, 'num_tokens': 18887096.0, 'mean_token_accuracy': 0.6162941067115121, 'epoch': 2.0})

In [144]:
trainer.model.save_pretrained("qwen3-punchlines-lora")
tokenizer.save_pretrained("qwen3-punchlines-lora")

('qwen3-punchlines-lora\\tokenizer_config.json',
 'qwen3-punchlines-lora\\special_tokens_map.json',
 'qwen3-punchlines-lora\\chat_template.jinja',
 'qwen3-punchlines-lora\\vocab.json',
 'qwen3-punchlines-lora\\merges.txt',
 'qwen3-punchlines-lora\\added_tokens.json',
 'qwen3-punchlines-lora\\tokenizer.json')

### Predict

In [ ]:
base_name = "Qwen/Qwen3-0.6B"
tok = AutoTokenizer.from_pretrained(base_name, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

dtype = torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.is_bf16_supported()) else (
    torch.float16 if torch.cuda.is_available() else torch.float32
)

base = AutoModelForCausalLM.from_pretrained(base_name, dtype=dtype).eval()


model = PeftModel.from_pretrained(base, "qwen3-punchlines-lora").eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_punchline(setup: str, max_new_tokens: int = 80) -> str:
    setup = setup.strip()
    inputs = tok([setup], return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,
            top_p=0.9,
            repetition_penalty=1.08,
        )
    gen_ids = out[0][inputs["input_ids"].shape[1]:]
    text = tok.decode(gen_ids, skip_special_tokens=True)
    # берём первую строку/фразу как “пончлайн”, чтобы не разгонялась
    text = text.strip().split("\n")[0].strip()
    return text

print(generate_punchline("Идёт мужик по лесу"))


— срать хочет, но не может - почему? Потому что он гей. Он в кустах сидит и думает: "Если я утоплюсь, то будет африка" Идет лес, и видит, а на дереве стоит другой мужик, и говорит: "Деньги нужны, но топить


In [190]:
rows = []

for item in tqdm(prefixes, total=len(prefixes), desc="Generating punchlines"):
    setup_id, setup_text = split_id_and_text(item)

    setup_text = setup_text.strip()
    if not setup_text:
        continue

    out = generate_punchline(setup_text)
    rows.append({"id": setup_id, "setup": setup_text, "punchline": out})

Generating punchlines: 100%|██████████| 75/75 [07:10<00:00,  5.74s/it]


In [175]:
data = pd.DataFrame(rows)
data.head()

,id,setup,punchline
0,1,Идёт мужик по лесу,"и видит — крокодил сидит Он подумал: ""Хорошо,..."
1,2,Встречаются два друга,"— Дорогая, ты вчера была на свадьбе! - Ой, я ..."
2,3,Приходят мужик в бар,"— Дорогой, ты не хочешь меня с женой? - Нет! ..."
3,4,Жена говорит мужу,"в постели: — Дорогой, ты мне не присножался!-..."
4,5,"Приходят альфа, бета и гамма в бар","Альфы скидывают 20%, бета — 50%, а гамма - 75..."


In [ ]:
data.to_excel("anecdotes.xlsx")